In [6]:
import re
import docx

In [ ]:
def openingfile(filename):
    wordlist = []
    doc = docx.Document(filename)
    for i in range(1, len(doc.paragraphs)):
        for j in range(len(doc.paragraphs[i].runs)):
            if doc.paragraphs[i].runs[j].bold == True:
                if len(doc.paragraphs[i].runs[j].text) > 1:
                    wordlist.append(doc.paragraphs[i].runs[j].text)
    return wordlist

In [ ]:
def wordlistfile(wl):
    plain = '\n'.join(wl)
    with open('Wordlist.txt', 'w', encoding='utf-8') as f:
        f.write(plain)

In [ ]:
def afan():
    clitics = []
    with open('Wordlist.txt', 'r', encoding='utf-8') as f:
        file = f.readlines()
    for item in file:
        item.lower()
        if item.startswith('af') or item.startswith('an'):
            clitics.append(item)
    return clitics

In [ ]:
def afanfile(clitics):
    file = ''.join(clitics)
    with open('afan.txt', 'w', encoding='utf-8') as f:
        f.write(file)

In [ ]:
## здесь я забираю из словаря все слова, которые начинаются на af- и an-
wl = openingfile('WB_s.docx')
wordlistfile(wl)
cli = afan()
afanfile(cli)

In [14]:
greek = re.compile(u'[\u1F00-\u1FFE\u0391-\u03CF\u03F4\u0390]', re.UNICODE)
def make_wb_better(greek):
    doc = docx.Document('WB_s.docx')
    fulltext = []
    for para in doc.paragraphs:
        parat = para.text
        paratext = []
        if re.search(greek, parat):
            for run in para.runs:
                if run.bold == True:
                    goth_w = run.text
                    paratext.append(goth_w)
                elif re.search(greek, run.text):
                    greek_w = run.text
                    paratext.append(greek_w)
        fulltext.append(paratext)
    new_list = []
    for paratext in fulltext:
        tabd = '\t'.join(paratext)
        new_list.append(tabd)
    return '\n'.join(new_list)

In [ ]:
def make_wb_txt(text):
    with open('wb_try.txt', 'w', encoding='utf-8') as f:
        f.write(text)

text = make_wb_better(greek)
make_wb_txt(text)

In [9]:
from math import fabs
greek = re.compile(u'[\u1F00-\u1FFE\u0391-\u03CF\u03F4\u0390]', re.UNICODE)
def clean_wordlist(greek):
    with open('wb_try.txt', 'r', encoding='utf-8') as f:
        text = f.read()
    text = re.sub('[0-9.,*:]', '', text)
    text = re.sub('\s+([\n\t]\S)', r'\1', text)
    text = re.sub('\n', '\t', text)
    text = re.sub('(\s)[A-Z][ηα]?\s+', r'\1', text)
    blocks = text.split('\t')
    new_text = ''
    for blo in blocks:
        block = blo.strip()
        if block:
            n_grk = len(greek.findall(block))
            n_ltn = len(re.findall('[A-Za-zþ]', block))
            if fabs(n_grk - n_ltn) > 2:
                if n_ltn > n_grk:
                    new_text += '\n'
                    new_text += block
                else:
                    new_text += '\t'
                    new_text += block
            elif len(block) <= 3:
                if re.match(r'[A-Z\u0391-\u03A9\u03F4]', block):
                    new_text += ''
                elif re.search('[a-zþ]', new_text[len(new_text)-3:]):
                    if n_grk == 0 and re.search('[a-zþ]', block):
                        new_text += block
                    elif n_ltn == 0 and n_grk > 0:
                        new_text += '\t'
                        new_text += block
                elif greek.search(new_text[len(new_text)-3:]):
                    if n_grk == 0 and re.search('[a-zþ]', block):
                        new_text += '\n'
                        new_text += block
                    elif n_ltn == 0 and n_grk > 0:
                        new_text += ' '
                        new_text += block
            else:
                print('последнее в буфере:')
                print(new_text[len(new_text)-5:])
                print('current block:')
                print(block)
                a = str(input('new_text += '))
                while a != '':
                    if a == '1':
                        new_text += block
                    else:
                        new_text += a
                    a = str(input('new_text += '))
    with open('wb_try2.txt', 'w', encoding='utf-8') as f2:
        f2.write(new_text)
    return new_text
                    
#better_wb = clean_wordlist(greek)

In [ ]:
def look_for_prefix():
    with open('wb_try2.txt', 'r', encoding='utf-8') as f:
        ftext = f.read()
    better = ftext[1:]
    better = re.sub('-\s?-', '', better)
    words = better.split('\n')
    with_prefix = {}
    for article in words:
        article = re.sub(r'\[', '', article)
        article = re.sub(r'\]', '', article)        
        art = article.split('\t', maxsplit=1)
        if re.match('[Aa][nf]-?', art[0]):
            try:
                with_prefix[art[0]] = art[1].split('\t')
            except IndexError:
                pass
    return with_prefix
        
with_prefix = look_for_prefix()

In [ ]:
def sort_prefixes(wb):
    pfxs = {}
    for word in sorted(wb.keys()):
        if '-' in word:
            pref = word.split('-')[0]
            transl = wb[word]
            if pref in pfxs:
                pfxs[pref].extend(transl)
            else:
                pfxs[pref] = transl
    with open('greek_pref.txt', 'r', encoding='utf-8') as g:
        g2 = g.read()
        pf_liste = re.split('-\n', g2)
    pfxs_up = {}
    for key in pfxs:
        for pf in pf_liste:
            pfxs_up[key] = {pf: 0}
    for key in pfxs:
        for token in pfxs[key]:
            if not token.startswith('('):
                repl = "[’''^<>-]"
                token = re.sub(repl, '', token)
                winlist = []
                if len(token) > 3:
                    if ' ' not in token[:4]:
                        tword = token
                    else:
                        tword = max(token.split(), key=len)
                    i = 4
                    while i > 0:
                        if tword[:i] in pf_liste:
                            winlist.append(tword[:i])
                        i -= 1
                    if winlist:
                        for pf in winlist:
                            if pf in pfxs_up[key]:
                                pfxs_up[key][pf] += 1
                            else:
                                pfxs_up[key][pf] = 1
                    else:
                        pf = '??' + tword[:3]
                        if pf in pfxs_up[key]:
                            pfxs_up[key][pf] += 1
                        else:
                            pfxs_up[key][pf] = 1
    with open('prfxs_use.txt', 'w', encoding='utf-8') as uf:
        for kee in pfxs_up:
            uf.write(kee + '-\n')
            ind = 1
            for pf in sorted(pfxs_up[kee], key=pfxs_up[kee].get, reverse=True)[:10]:
                uf.write(str(ind) + '\t')
                uf.write(pf + '\t' + str(pfxs_up[kee][pf]) + '\n')
                ind += 1
    return pfxs_up       
    
pfxs_up = sort_prefixes(with_prefix)

In [3]:
def look_for_piu():
    with open('wb_try2.txt', 'r', encoding='utf-8') as f:
        ftext = f.read()
    better = ftext[1:]
    better = re.sub('-\s?-', '', better)
    words = better.split('\n')
    found = {}
    for article in words:
        article = re.sub(r'\[', '', article)
        article = re.sub(r'\]', '', article)        
        art = article.split('\t', maxsplit=1)
        if re.match('þiu?', art[0]):
            try:
                found[art[0]] = art[1].split('\t')
            except IndexError:
                pass
    with open('piupiu.txt', 'w', encoding='utf-8') as pfp:
        for item in sorted(found.keys()):
            pfp.write(item + '\t-->\t' + ';'.join(found[item]))
            pfp.write('\n')

look_for_piu()